In [1]:
import pandas as pd

In [2]:
train = pd.read_csv(r'C:\Users\ASUS\Downloads\fake-news\train.csv')

In [3]:
train.title[5]

'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart'

In [4]:
train.dropna(inplace=True)

In [5]:
train.reset_index(inplace=True)

In [6]:
train.drop(['index','id'],1,inplace=True)

In [7]:
train_data = train.drop('label',1)

In [8]:
train_label = train['label']

In [9]:
train_data.shape

(18285, 3)

In [10]:
train_label.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.1.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
voc_size=5000

In [15]:
message = train['title']

In [16]:
message

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.stem import WordNetLemmatizer
ps = WordNetLemmatizer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [22]:
one_hot_rep = [one_hot(words,voc_size) for words in corpus]

In [23]:
one_hot_rep

[[1735, 1302, 2199, 4093, 4610, 946, 1472, 323, 928, 888],
 [4078, 2560, 2636, 2683, 1745, 3287, 4187],
 [3933, 1272, 2481, 2623],
 [1867, 3129, 3743, 508, 646, 642],
 [1617, 1745, 374, 2680, 4433, 4129, 1745, 3823, 2543, 3187],
 [4145,
  221,
  4744,
  3817,
  4809,
  1149,
  3321,
  1639,
  3482,
  507,
  2704,
  2186,
  4191,
  431,
  4187],
 [4846, 331, 4799, 3872, 4597, 1968, 1231, 2448, 4650, 3430, 3223],
 [1928, 3419, 3368, 2466, 2174, 4475, 1149, 958, 4650, 3430, 3223],
 [68, 1370, 2394, 4677, 4526, 3405, 128, 1856, 1149, 4937],
 [4179, 284, 422, 3407, 1768, 3362, 3596, 66],
 [845, 4006, 4221, 30, 2650, 2036, 3146, 1471, 4953, 4485, 3795],
 [508, 1925, 4610, 142, 1149, 2174],
 [4084, 4990, 770, 4688, 2692, 278, 4514, 121, 1883],
 [3091, 402, 2034, 2830, 4687, 4634, 2869, 4650, 3430, 3223],
 [783, 1955, 4725, 81, 526, 4650, 3430, 3223],
 [3418, 932, 2028, 332, 1475, 2123, 1831, 4068, 1147, 3470],
 [3951, 1071, 2560],
 [659, 1304, 1357, 97, 1149, 2803, 1440, 4187],
 [2104, 456, 2

In [24]:
sent_length=20
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  323  928  888]
 [   0    0    0 ... 1745 3287 4187]
 [   0    0    0 ... 1272 2481 2623]
 ...
 [   0    0    0 ... 4650 3430 3223]
 [   0    0    0 ... 4951 1906 3842]
 [   0    0    0 ... 4588 1391  774]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1735,
       1302, 2199, 4093, 4610,  946, 1472,  323,  928,  888])

In [26]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(train_label)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 7s 577us/sample - loss: 0.3397 - accuracy: 0.8418 - val_loss: 0.1961 - val_accuracy: 0.9196
Epoch 2/10
12250/12250 [==============================] - 4s 342us/sample - loss: 0.1366 - accuracy: 0.9471 - val_loss: 0.2038 - val_accuracy: 0.9148
Epoch 3/10
12250/12250 [==============================] - 4s 343us/sample - loss: 0.0897 - accuracy: 0.9666 - val_loss: 0.2241 - val_accuracy: 0.9152
Epoch 4/10
12250/12250 [==============================] - 4s 320us/sample - loss: 0.0624 - accuracy: 0.9779 - val_loss: 0.2800 - val_accuracy: 0.9148
Epoch 5/10
12250/12250 [==============================] - 4s 313us/sample - loss: 0.0402 - accuracy: 0.9860 - val_loss: 0.2900 - val_accuracy: 0.9137
Epoch 6/10
12250/12250 [==============================] - 4s 328us/sample - loss: 0.0266 - accuracy: 0.9913 - val_loss: 0.3836 - val_accuracy: 0.9070
Epoch 7/10
12250/12250 [===========================

In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 4s 330us/sample - loss: 0.0045 - accuracy: 0.9988 - val_loss: 0.5229 - val_accuracy: 0.9070
Epoch 2/10
12250/12250 [==============================] - 4s 334us/sample - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.6012 - val_accuracy: 0.9051
Epoch 3/10
12250/12250 [==============================] - 5s 374us/sample - loss: 0.0058 - accuracy: 0.9982 - val_loss: 0.5762 - val_accuracy: 0.9017
Epoch 4/10
12250/12250 [==============================] - 4s 361us/sample - loss: 0.0079 - accuracy: 0.9973 - val_loss: 0.7198 - val_accuracy: 0.9049
Epoch 5/10
12250/12250 [==============================] - 4s 348us/sample - loss: 0.0055 - accuracy: 0.9979 - val_loss: 0.6354 - val_accuracy: 0.9080
Epoch 6/10
12250/12250 [==============================] - 4s 353us/sample - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.6520 - val_accuracy: 0.8984
Epoch 7/10
12250/12250 [===========================

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model_dropout=Sequential()
model_dropout.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model_dropout.add(Dropout(0.3))
model_dropout.add(LSTM(100))
model_dropout.add(Dropout(0.3))
model_dropout.add(Dense(1,activation='sigmoid'))
model_dropout.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
y_pred_dropout=model.predict_classes(X_test)